In [1]:
import importlib, sys
import pickle
import numpy as np
import warnings
from sklearn.metrics import mean_squared_error
import pandas as pd
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
parentdir = 'C:\\Users\\danie\\Documents\\work\\testingRS\\TESTS'
sys.path.insert(0,parentdir) 
import entity
importlib.reload(entity)
from entity import User, CB, MF, CF, HYBRID
import utils.dataset_metrics as dt

In [186]:
with open('C:\\Users\\danie\\Documents\\work\\testingRS\\TESTS\\objects\\tfidf.pkl', 'rb') as f:
    tfidf_matrix, tfidf_feature_names, full_item_ids = pickle.load(f)
    
with open('C:\\Users\\danie\\Documents\\work\\testingRS\\TESTS\\objects\\subset_1000.pkl', 'rb') as f:
    utility_matrix_1, interactions_work_1 = pickle.load(f)
    
with open('C:\\Users\\danie\\Documents\\work\\testingRS\\TESTS\\objects\\subset_40000.pkl', 'rb') as f:
    utility_matrix_2, interactions_work_2 = pickle.load(f)
    
with open('C:\\Users\\danie\\Documents\\work\\testingRS\\TESTS\\objects\\subset_72269.pkl', 'rb') as f:
    utility_matrix_3, interactions_work_3 = pickle.load(f)

## FILTER 1 - Users with al least 10 interactions

In [278]:
count_interactions = interactions_work_1.groupby(['personId', 'contentId']).size().groupby('personId').size()
filtered_df1 = count_interactions[count_interactions >= 10].reset_index()[['personId']]
interactions_subset_users = interactions_work_1[interactions_work_1['personId'].isin(list(filtered_df1.personId))]
sub_um1 = interactions_subset_users.pivot(index='personId', columns='contentId', values='rating').fillna(0)
print("SPARSITY = {}".format(dt.get_overall_sparsity(sub_um1)))
print("USUARIOS = {} / ITEMS = {}".format(sub_um1.shape[0], sub_um1.shape[1]))

SPARSITY = 0.8223
USUARIOS = 12 / ITEMS = 98


## FILTER 2 - Items with at least 5 interactions

In [184]:
items_interactions = interactions_work_1.groupby(['personId', 'contentId']).size().groupby('contentId').size()
filtered_df2 = items_interactions[items_interactions >= 5].reset_index()[['contentId']]
interactions_subset_items = interactions_work_1[interactions_work_1['contentId'].isin(list(filtered_df2.contentId))]
sub_um2 = interactions_subset_items.pivot(index='personId', columns='contentId', values='rating').fillna(0)
print("SPARSITY = {}".format(dt.get_overall_sparsity(sub_um2)))
print("USUARIOS = {} / ITEMS = {}".format(sub_um2.shape[0], sub_um2.shape[1]))

SPARSITY = 0.9576
USUARIOS = 211 / ITEMS = 59


## FILTER 3 - BOTH

In [342]:
interactions_subset = interactions_work_1[interactions_work_1['personId'].isin(list(filtered_df1.personId))
                                          & interactions_work_1['contentId'].isin(list(filtered_df2.contentId))]
sub_um3 = interactions_subset.pivot(index='personId', columns='contentId', values='rating').fillna(0)
print("SPARSITY = {}".format(dt.get_overall_sparsity(sub_um3)))
print("USUARIOS = {} / ITEMS = {}".format(sub_um3.shape[0], sub_um3.shape[1]))

SPARSITY = 0.7676
USUARIOS = 12 / ITEMS = 52


# PRUEBAS / SUB1

In [187]:
um = sub_um1
interactions = interactions_subset_users
# um.shape[1] - (um == 0).astype(int).sum(axis=1) ## INTERACCIONES POR PERSONA


In [191]:
USER1 = '-1032019229384696495'
USER2 = '1623838599684589103'
USER3 = '4340306774493623681'

FACTORS_MF = 5
NUMBER_NEIGHBORS = 10


In [201]:
M1 = dt.get_metric1(USER3,um)
print(M1)
M3 = dt.get_metric3(um) ###EL 5% ES MENOR QUE UNO. SIEMPRE VA A DAR 1
print(M3)


0.306


In [146]:
user_object = User(USER1, um, interactions)
mf = MF(user_object, FACTORS_MF)
mf_predictions = mf.predictions_user()
mf_predictions

contentId,-1021685224930603833,-1672166631728511207,-1802980374508081539,-2711301039947937868,-2778760500673113802,-327208414896292296,-36844712915106340,-3810375613673461625,-4102297002729307038,-4571929941432664145,...,7748594134538052724,787067277772219433,8095004770374551394,809601605585939618,8194079557551008273,8219310215587599928,8471417198703153120,8545647269051113523,8565115980515859979,9032993320407723266
-1032019229384696495,0.120919,0.133445,0.121166,0.145162,0.109569,0.10444,0.124964,0.13621,0.088982,0.143434,...,0.108964,0.096494,0.133054,0.11919,0.120919,0.120314,0.175409,0.120314,0.13621,0.145162


# PRUEBAS / SUB2

In [202]:
um = sub_um2
interactions = interactions_subset_items
um.shape[1] - (um == 0).astype(int).sum(axis=1) ## INTERACCIONES POR PERSONA


personId
-1032019229384696495    20
-108842214936804958     12
-123314413156450014      1
-1352542225971050638     2
-1387464358334758758     2
                        ..
9038446466275805109      1
9050204922960952289      2
9135582630122950040      3
9137372837662939523      1
9210530975708218054      1
Length: 211, dtype: int64

In [203]:
USER1 = '-1032019229384696495'
USER2 = '-108842214936804958'
USER3 = '-1387464358334758758'

FACTORS_MF = 5
NUMBER_NEIGHBORS = 10

In [206]:
M1 = dt.get_metric1(USER3,um)
print(M1)
M3 = dt.get_metric3(um) ###EL 5% ES 10
print(M3)

0.1
0.254


In [165]:
user_object = User(USER3, um, interactions)
mf = MF(user_object, FACTORS_MF)
mf_predictions = mf.predictions_user()
mf_predictions

contentId,-1021685224930603833,-1039912738963181810,-1590585250246572231,-1672166631728511207,-2081760549863309770,-2176468683077766369,-3173020603774823976,-36844712915106340,-4102297002729307038,-4110991218639855802,...,7520301770873472812,7528802258213768379,787067277772219433,8095004770374551394,809601605585939618,8194079557551008273,8471417198703153120,8742078838645536785,9032993320407723266,9122627895188486603
-1387464358334758758,0.172501,0.178193,0.188813,0.178107,0.1769,0.178188,0.20151,0.167641,0.172535,0.180748,...,0.173172,0.167631,0.171052,0.168437,0.167192,0.171574,0.169833,0.189952,0.17273,0.185148


# PRUEBAS / SUB3

In [207]:
um = sub_um3
interactions = interactions_subset
um.shape[1] - (um == 0).astype(int).sum(axis=1) ## INTERACCIONES POR PERSONA


personId
-1032019229384696495    20
-108842214936804958     12
-1443636648652872475    10
-2511855597392146401    16
-2626634673110551643    13
-7267769888748948232     6
1120069409160402054     11
1623838599684589103     15
3609194402293569455     13
3891637997717104548     13
4340306774493623681      5
8414731042150985013     11
dtype: int64

In [208]:
USER1 = '-1032019229384696495'
USER2 = '-108842214936804958'
USER3 = '4340306774493623681'

FACTORS_MF = 5
NUMBER_NEIGHBORS = 10

In [211]:
M1 = dt.get_metric1(USER3,um)
print(M1)
M3 = dt.get_metric3(um) ###EL 5% ES MENOR QUE UNO. SIEMPRE VA A DAR 1
print(M3)

0.25
1.0


In [180]:
user_object = User(USER3, um, interactions)
mf = MF(user_object, FACTORS_MF)
mf_predictions = mf.predictions_user()
mf_predictions

contentId,-1021685224930603833,-1039912738963181810,-1590585250246572231,-1672166631728511207,-2176468683077766369,-36844712915106340,-4102297002729307038,-4110991218639855802,-454649054276160610,-4571929941432664145,...,6152652267138213180,6157037646878010131,7520301770873472812,787067277772219433,8095004770374551394,809601605585939618,8194079557551008273,8742078838645536785,9032993320407723266,9122627895188486603
4340306774493623681,0.141946,0.208899,0.178297,0.141432,0.217737,0.122753,0.148611,0.118889,0.167214,0.058612,...,0.180143,0.141946,0.141946,0.129239,0.093747,0.176586,0.141946,0.133181,0.108039,0.106309


## NEW USER - NO INTERACTIONS

In [3]:
NEW_USER = '-1714802647008415227'

### Filter 1

In [302]:
# utiliity matrix
listofzeros = [0] * sub_um1.shape[1]
row_df = pd.DataFrame([listofzeros], index = [NEW_USER], columns =  sub_um1.columns)
sub_um1_n = sub_um1.append(row_df)

## user object
user_object = User(NEW_USER, sub_um1_n, interactions_subset_users)
mf = MF(user_object, FACTORS_MF)
mf_predictions = mf.predictions_user()

#metrics
M1 = dt.get_metric1(NEW_USER,sub_um1_n)
print(M1)
M3 = dt.get_metric3(sub_um1_n) 
print(M3)

mf_predictions

0.0
1.0


contentId,-1021685224930603833,-1039912738963181810,-1590585250246572231,-1672166631728511207,-1802980374508081539,-2081760549863309770,-2176468683077766369,-2711301039947937868,-2778760500673113802,-3173020603774823976,...,8095004770374551394,809601605585939618,8194079557551008273,8219310215587599928,8471417198703153120,8545647269051113523,8565115980515859979,8742078838645536785,9032993320407723266,9122627895188486603
-1714802647008415227,0.119462,0.119462,0.119462,0.119462,0.119462,0.119462,0.119462,0.119462,0.119462,0.119462,...,0.119462,0.119462,0.119462,0.119462,0.119462,0.119462,0.119462,0.119462,0.119462,0.119462


## Filter 2

In [309]:
# utiliity matrix
listofzeros = [0] * sub_um2.shape[1]
row_df = pd.DataFrame([listofzeros], index = [NEW_USER], columns =  sub_um2.columns)
sub_um2_n = sub_um2.append(row_df)

## user object
user_object = User(NEW_USER, sub_um2_n, interactions_subset_items)
mf = MF(user_object, FACTORS_MF)
mf_predictions = mf.predictions_user()


#metrics
M1 = dt.get_metric1(NEW_USER,sub_um2_n)
print(M1)
M3 = dt.get_metric3(sub_um2_n) 
print(M3)

mf_predictions

0.0
0.254


contentId,-1021685224930603833,-1039912738963181810,-1590585250246572231,-1672166631728511207,-2081760549863309770,-2176468683077766369,-3173020603774823976,-36844712915106340,-4102297002729307038,-4110991218639855802,...,7520301770873472812,7528802258213768379,787067277772219433,8095004770374551394,809601605585939618,8194079557551008273,8471417198703153120,8742078838645536785,9032993320407723266,9122627895188486603
-1714802647008415227,0.170708,0.170708,0.170708,0.170708,0.170708,0.170708,0.170708,0.170708,0.170708,0.170708,...,0.170708,0.170708,0.170708,0.170708,0.170708,0.170708,0.170708,0.170708,0.170708,0.170708


## Filter 3

In [306]:
# utiliity matrix
listofzeros = [0] * sub_um3.shape[1]
row_df = pd.DataFrame([listofzeros], index = [NEW_USER], columns =  sub_um3.columns)
sub_um3_n = sub_um3.append(row_df)

## user object
user_object = User(NEW_USER, sub_um3_n, interactions_subset)
mf = MF(user_object, FACTORS_MF)
mf_predictions = mf.predictions_user()

#metrics
M1 = dt.get_metric1(NEW_USER,sub_um3_n)
print(M1)
M3 = dt.get_metric3(sub_um3_n) 
print(M3)

mf_predictions

0.0
1.0


contentId,-1021685224930603833,-1039912738963181810,-1590585250246572231,-1672166631728511207,-2081760549863309770,-2176468683077766369,-3173020603774823976,-36844712915106340,-4102297002729307038,-4110991218639855802,...,6157037646878010131,7520301770873472812,787067277772219433,8095004770374551394,809601605585939618,8194079557551008273,8471417198703153120,8742078838645536785,9032993320407723266,9122627895188486603
-1714802647008415227,0.122087,0.122087,0.122087,0.122087,0.122087,0.122087,0.122087,0.122087,0.122087,0.122087,...,0.122087,0.122087,0.122087,0.122087,0.122087,0.122087,0.122087,0.122087,0.122087,0.122087


## NEW ITEM - NO INTERACTIONS

In [356]:
NEW_ITEM = '-1591454024897803197'
USER = '8414731042150985013'

In [ ]:
-108842214936804958	
4340306774493623681	
8414731042150985013	

0.122087	

In [357]:
sub_um3n2 = sub_um3.copy()
listofzeros = [0] * sub_um3.shape[0]
col_df = pd.DataFrame(index = sub_um3.index, columns = [NEW_ITEM])
col_df[NEW_ITEM] = listofzeros
sub_um3n2.insert(0, NEW_ITEM, listofzeros)

## user object
user_object = User(USER, sub_um3n2, interactions_subset)
mf = MF(user_object, FACTORS_MF)
mf_predictions = mf.predictions_user()

#metrics
M1 = dt.get_metric1(USER,sub_um3n2)
print(M1)
M3 = dt.get_metric3(sub_um3n2) 
print(M3)

mf_predictions

0.55
0.981


contentId,-1591454024897803197,-1021685224930603833,-1039912738963181810,-1590585250246572231,-1672166631728511207,-2081760549863309770,-2176468683077766369,-3173020603774823976,-36844712915106340,-4102297002729307038,...,6152652267138213180,6157037646878010131,7520301770873472812,787067277772219433,8095004770374551394,8194079557551008273,8471417198703153120,8742078838645536785,9032993320407723266,9122627895188486603
8414731042150985013,0.122087,0.104961,0.261072,0.273893,0.129688,0.177487,0.247627,0.121336,0.281677,0.162835,...,0.18484,0.104961,0.104961,0.102858,0.213983,0.104961,0.143843,0.206376,0.156296,0.238738


## pruebas MF - input object mf

In [181]:
items_test = mf.user.items_to_ignore
df_res = pd.DataFrame(index=list(items_test), columns=['ORIGINAL', 'MF'])
mf_hits= []
for i, IT in enumerate(items_test):
    print("{} of {} items".format(i, len(items_test)))
    new_utility = mf.user.utility_matrix.copy()
    new_utility.loc[mf.user.id, IT] = 0.0
    new_interactions = mf.user.interactions_df.drop(mf.user.interactions_df[(mf.user.interactions_df.personId == mf.user.id) & (mf.user.interactions_df.contentId == IT)].index)
    new_user_object = User(mf.user.id, new_utility, new_interactions)
    nmf = MF(new_user_object, FACTORS_MF)
    nmf_predictions = nmf.predictions_user()
    top10 = list(nmf_predictions.sort_values(by=[nmf.user.id], axis=1, ascending=False).iloc[0, 0:10].index)
    df_res.loc[IT, 'ORIGINAL'] = mf.user.utility_matrix.loc[mf.user.id, IT]/5.0
    df_res.loc[IT, 'MF'] = nmf_predictions[IT].values[0]
    if IT in top10:
        mf_hits.append(IT)

0 of 5 items
1 of 5 items
2 of 5 items
3 of 5 items
4 of 5 items


In [182]:
print("HITS = {}".format(len(mf_hits)))

HITS = 3


In [183]:
df_res
print(round(mean_squared_error(df_res['ORIGINAL'].values, df_res['MF'].values),3))

0.06


## ONE INTERACTION

In [4]:
USER = "9210530975708218054'"

In [5]:
user_object = User(USER, utility_matrix_1, interactions_work_1)
hy = HYBRID(user_object,FACTORS_MF, NUMBER_NEIGHBORS, full_item_ids, tfidf_matrix)

NameError: name 'utility_matrix_1' is not defined